*This notebook requires fenicstools to generate the DOFs plots* 

In [1]:
import dolfin as df
import numpy as np

In [2]:
import finmag

[2015-04-15 15:28:49] INFO: Finmag logging output will be appended to file: '/home/david/.finmag/global.log'
[2015-04-15 15:28:49] DEBUG: Building modules in 'native'...
[2015-04-15 15:28:50] DEBUG: FinMag          5797:b416a2672cfee657ac9e9aaf8d69f7daeb55227d
[2015-04-15 15:28:50] DEBUG: Dolfin          1.5.0                Matplotlib      lazily loaded       
[2015-04-15 15:28:50] DEBUG: Numpy           1.9.2                Scipy           0.13.3              
[2015-04-15 15:28:50] DEBUG: IPython         3.1.0                Python          2.7.6               
[2015-04-15 15:28:50] DEBUG: Paraview        4.0.1-1ubuntu1       Sundials        2.5.0               
[2015-04-15 15:28:50] DEBUG: Boost-Python    <unknown>            Linux           Ubuntu 14.04.2 LTS  
[2015-04-15 15:28:50] DEBUG: Registering debug signal handler. Press Ctrl-Z any time to stop execution and jump into the debugger.


Generate a 2D sqaure mesh:

In [3]:
mesh = df.RectangleMesh(-4, -4, 4, 4, 4, 4)

In [4]:
df.plot(mesh, interactive=True)

<dolfin.cpp.io.VTKPlotter; proxy of <Swig Object of type 'std::shared_ptr< dolfin::VTKPlotter > *' at 0x7fc417e87780> >

Mesh coordinates:

In [5]:
mesh.coordinates()

array([[-4., -4.],
       [-2., -4.],
       [ 0., -4.],
       [ 2., -4.],
       [ 4., -4.],
       [-4., -2.],
       [-2., -2.],
       [ 0., -2.],
       [ 2., -2.],
       [ 4., -2.],
       [-4.,  0.],
       [-2.,  0.],
       [ 0.,  0.],
       [ 2.,  0.],
       [ 4.,  0.],
       [-4.,  2.],
       [-2.,  2.],
       [ 0.,  2.],
       [ 2.,  2.],
       [ 4.,  2.],
       [-4.,  4.],
       [-2.,  4.],
       [ 0.,  4.],
       [ 2.,  4.],
       [ 4.,  4.]])

In [6]:
S1 = df.FunctionSpace(mesh, 'CG', 1)
S3 = df.VectorFunctionSpace(mesh, 'CG', 1, dim=3)

In [10]:
v_fun = df.Function(S3)

Simulation with PBCs:

In [7]:
Ms = 1e5

In [8]:
sim = finmag.Simulation(mesh, Ms, unit_length=1e-9, pbc='2d')

[2015-04-15 15:29:04] INFO: Finmag logging output will be written to file: '/home/david/tmp/PBC_load_test/unnamed.log' (any old content will be overwritten).
[2015-04-15 15:29:04] DEBUG: Creating DataWriter for file 'unnamed.ndt'
[2015-04-15 15:29:04] INFO: Creating Sim object 'unnamed' (rank=0/1).
[2015-04-15 15:29:04] INFO: <Mesh of topological dimension 2 (triangles) with 25 vertices and 32 cells, ordered>
[2015-04-15 15:29:04] DEBUG: Setting 2d periodic boundary conditions (in the xy-plane).
[2015-04-15 15:29:04] DEBUG: Creating LLG object.


The values in the boundaries are (0, 0, -1) while the rest of the mesh vertexes will have associated a value of (0, 0, 1)

In [9]:
def m_init(pos):
    x, y = pos[0], pos[1]
    if x > 3 or x < -3:
        return (0, 0, -1)
    elif y > 3 or y < -3:
        return (0, 0, -1)
    else:
        return (0, 0, 1)
    

In [10]:
sim.set_m(m_init)

Now we will try to visualize the DOFs ordering and the mesh numbering:

In [46]:
%matplotlib qt4

In [42]:
import fenicstools

We can plot the DOFs (degrees of freedom) of the 3 components of the vector field (in the plot windows, press `D` to get the DOFs and `V` to get the vertex numbering). The values are the array indexes of the vector field, which goes into the mesh in the right order, following the **vertex to dof** mapping (see below):

In [60]:
dmp = fenicstools.DofMapPlotter(sim.S3)
dmp.plot([0, 1, 2])
dmp.show()

 
Figure 1 : Plotting dofmaps [0, 1, 2] 


<img src='dofs_of_0_1_2.png' width=700>

Some values are repeated since they are the PBCs

The mesh indexes:

<img src='dofs_vertexes.png' width=500>

The dolfin vector function (reduced):

In [65]:
sim.m_field.f.vector().array()

array([ 0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  1.,  0.,  0., -1.,  0.,
        0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,
        1.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,
        0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  1.])

The Vertex to DOF map give us the mapping for going from the dofin function towards the mesh ordering, which is a **full length** vector (the system including boundaries). It automatically copies the repeated values at the boundaries with PBC:

In [64]:
v2d = df.vertex_to_dof_map(sim.S3)
v2d

array([ 0,  3,  6,  1,  4,  7, 12, 14, 16, 18, 20, 22,  0,  3,  6,  9, 10,
       11,  2,  5,  8, 13, 15, 17, 19, 21, 23,  9, 10, 11, 27, 28, 29, 24,
       25, 26, 30, 31, 32, 33, 34, 35, 27, 28, 29, 39, 40, 41, 36, 37, 38,
       42, 43, 44, 45, 46, 47, 39, 40, 41,  0,  3,  6,  1,  4,  7, 12, 14,
       16, 18, 20, 22,  0,  3,  6], dtype=int32)

This gives us the dolfin function values in the same order than the mesh, in the *xyz* order

$$ [\underbrace{x_0, y_0, z_0}_\text{node 0}, \underbrace{x_1, y_1, z_1}_\text{node 1}, \underbrace{x_2, y_2, z_2}_\text{node 2}, \ldots  ] $$


when calling a dolfin function with the mapping argument. The mapped function in array form looks like:

In [77]:
vfunction_to_mesh = sim.m_field.f.vector().array()[v2d]
vfunction_to_mesh

array([ 0.,  0., -1.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0., -1.,  0.,
        0., -1.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
        1.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  1.,
        0.,  0.,  1.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0.,  1.,  0.,
        0.,  1.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0.,
       -1.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0., -1.])

Since we know it is in the *xyz* order with respect to the mesh coordinates, we can print them according to their mesh node association: 

In [76]:
for i in xrange(len(v2d) / 3):
    print sim.mesh.coordinates()[i], ' --> ',  vfunction_to_mesh.reshape(-1, 3)[i]

[-4. -4.]  -->  [ 0.  0. -1.]
[-2. -4.]  -->  [ 0.  0. -1.]
[ 0. -4.]  -->  [ 0.  0. -1.]
[ 2. -4.]  -->  [ 0.  0. -1.]
[ 4. -4.]  -->  [ 0.  0. -1.]
[-4. -2.]  -->  [ 0.  0. -1.]
[-2. -2.]  -->  [ 0.  0.  1.]
[ 0. -2.]  -->  [ 0.  0.  1.]
[ 2. -2.]  -->  [ 0.  0.  1.]
[ 4. -2.]  -->  [ 0.  0. -1.]
[-4.  0.]  -->  [ 0.  0. -1.]
[-2.  0.]  -->  [ 0.  0.  1.]
[ 0.  0.]  -->  [ 0.  0.  1.]
[ 2.  0.]  -->  [ 0.  0.  1.]
[ 4.  0.]  -->  [ 0.  0. -1.]
[-4.  2.]  -->  [ 0.  0. -1.]
[-2.  2.]  -->  [ 0.  0.  1.]
[ 0.  2.]  -->  [ 0.  0.  1.]
[ 2.  2.]  -->  [ 0.  0.  1.]
[ 4.  2.]  -->  [ 0.  0. -1.]
[-4.  4.]  -->  [ 0.  0. -1.]
[-2.  4.]  -->  [ 0.  0. -1.]
[ 0.  4.]  -->  [ 0.  0. -1.]
[ 2.  4.]  -->  [ 0.  0. -1.]
[ 4.  4.]  -->  [ 0.  0. -1.]


The DOF to vertex map, which is reduced, is:

In [67]:
d2v = df.dof_to_vertex_map(sim.S3)
d2v

array([72, 63, 18, 73, 64, 19, 74, 65, 20, 27, 28, 29, 66, 21, 67, 22, 68,
       23, 69, 24, 70, 25, 71, 26, 33, 34, 35, 42, 43, 44, 36, 37, 38, 39,
       40, 41, 48, 49, 50, 57, 58, 59, 51, 52, 53, 54, 55, 56], dtype=uint64)

This is the opposite mapping, for going from the ordered mesh towards the *original function* (which has a reduced number of elements, since it does not consider the repeated boundaries)

In [78]:
vfunction_mesh_to_original = vfunction_to_mesh[d2v]
vfunction_mesh_to_original

array([ 0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  1.,  0.,  0., -1.,  0.,
        0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,
        1.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,
        0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  1.])

To compare this array, the original function was:

In [74]:
sim.m_field.f.vector().array()

array([ 0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  1.,  0.,  0., -1.,  0.,
        0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,
        1.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,
        0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  1.])